In [ ]:
import pandas as pd
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
plt.style.use('fivethirtyeight')

# Russell Ranch - Plant Growth Metrics

In [ ]:
# Survived data
all_plant_growth_df = pd.read_csv('Plant_Growth_Metrics.csv')

survived_plant_growth_df = all_plant_growth_df[all_plant_growth_df['seed_survival'] > 0]

all_plant_growth_df['seed_survival_rate'] = all_plant_growth_df['seed_survival_rate'] * 100
survived_plant_growth_df['seed_survival_rate'] = survived_plant_growth_df['seed_survival_rate'] * 100

all_plant_growth_df['shoot_length_cm'] = all_plant_growth_df['shoot_length_cm'].fillna(0)
all_plant_growth_df['num_spikelets'] = all_plant_growth_df['num_spikelets'].fillna(0)

In [ ]:
def plot_plant_growth(dataframe, data_label, title, x_label, y_label):
    sns.boxplot(x='treatment', y=data_label, data=dataframe, hue='soil')
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()

In [ ]:
# Seed survival could only be 0, 0.25, 0.5, 0.75, or 1, so graph doesn't seem very useful.
# plt.ylim(-10, 110)
# plt.yticks(range(0, 110, 25))
plot_plant_growth(all_plant_growth_df, 'seed_survival_rate', 'Seed Survival Rate per pot by Treatment', 'Treatment', 'Seed Survival %')

# Shoot length by treatment plot
plot_plant_growth(survived_plant_growth_df, 'shoot_length_cm', 'Shoot Length by Treatment (Only survived seeds)', 'Treatment', 'Shoot Length (cm)')


# Not very significant data
plot_plant_growth(survived_plant_growth_df, 'num_spikelets', 'Spikelet Count by Treatment (Only survived seeds)', 'Treatment', 'Spikelet Count')


# Root length + biomass in manteca has null values. think of how to address
# plot_plant_growth('root_length_cm', 'Root Length by Treatment', 'Treatment', 'Root Length (cm)')
# plot_plant_growth('shoot_root_biomass_g', 'Root + Shoot Biomass by Treatment', 'Treatment', 'Root + Shoot Biomass')

In [ ]:
chem_properties_df = pd.read_csv('Chemical_Properties.csv')

chem_properties_df['category'] = chem_properties_df['Soil'] + '_' + chem_properties_df['Treatment']

RR_chem_properties_df = chem_properties_df[chem_properties_df['Soil'] == "Russell Ranch"].dropna()
Man_chem_properties_df = chem_properties_df[chem_properties_df['Soil'] == "Manteca"].dropna()
t0_chem_properties_df = chem_properties_df[chem_properties_df['Time'] == "T0"].dropna()
end_chem_properties_df = chem_properties_df[chem_properties_df['Time'] == "end"].dropna()
# RR_ph_averages_df = RR_chem_properties_df[['Treatment', 'pH', 'Time']]
# RR_ph_averages_df.groupby(['Treatment', 'Time']).mean()
# RR_ph_diffs_df = RR_ph_averages_df['Treatment']

In [ ]:
def plot_chem_properties(data_label, title, x_label, y_label):
    fig, axes = plt.subplots(1, 2, figsize=(12,8))
    
    sns.boxplot(x='Treatment', y=data_label, data=RR_chem_properties_df, hue='Time', ax=axes[0])
    sns.boxplot(x='Treatment', y=data_label, data=Man_chem_properties_df, hue='Time', ax=axes[1])
    axes[0].set_title('Russell Ranch')
    axes[1].set_title('Manteca')
    fig.suptitle(title)
    
    axes[0].set_xlabel(x_label)
    axes[0].set_ylabel(y_label)
    
    axes[1].set_xlabel(x_label)
    axes[1].set_ylabel(y_label)
    
    axes[0].set_ylim(6.5, 9)
    axes[1].set_ylim(6.5, 9)
    plt.show()

In [ ]:
def plot_ph_properties(data_label, title, x_label, y_label):
    fig, axes = plt.subplots(1, 2, figsize=(12,8))
    sns.boxplot(x='Treatment', y=data_label, data=t0_chem_properties_df, hue='Soil', ax=axes[0])
    sns.boxplot(x='Treatment', y=data_label, data=end_chem_properties_df, hue='Soil', ax=axes[1])
    axes[0].set_title('T0')
    axes[1].set_title('9 Weeks')
    fig.suptitle(title)
    
    axes[0].set_xlabel(x_label)
    axes[0].set_ylabel(y_label)
    
    axes[1].set_xlabel(x_label)
    axes[1].set_ylabel(y_label)

    axes[0].set_ylim(6.5, 9)
    axes[1].set_ylim(6.5, 9)
    plt.show()

def plot_ec_properties(data_label, title, x_label, y_label):
    fig, axes = plt.subplots(1, 2, figsize=(12,8))
    sns.boxplot(x='Treatment', y=data_label, data=t0_chem_properties_df, hue='Soil', ax=axes[0])
    sns.boxplot(x='Treatment', y=data_label, data=end_chem_properties_df, hue='Soil', ax=axes[1])
    axes[0].set_title('T0')
    axes[1].set_title('9 Weeks')
    fig.suptitle(title)
    
    axes[0].set_xlabel(x_label)
    axes[0].set_ylabel(y_label)
    
    axes[1].set_xlabel(x_label)
    axes[1].set_ylabel(y_label)

    axes[0].set_ylim(-100, 1100)
    axes[1].set_ylim(-100, 1100)
    plt.show()

In [ ]:
plot_ph_properties('pH', "Treatment vs pH (per Soil Type), from T0 to End", "Treatment", "pH")
plot_ec_properties('EC (uS/cm)', "Treatment vs Electrical Conductivity (per Soil Type), from T0 to End", "Treatment", "EC (uS/cm)")

# Statistical Analysis

In [ ]:
# Logistic Regression for soil type and treatment for seed survival (since binary)
# Count regression for soil type and treatment for spikelet count
# Regression for soil type and treatment for shoot length

## Seed Survival Statistics

In [ ]:
# Ensure categorical columns are strings (sometimes they might have mixed types)
plant_growth_variables = all_plant_growth_df[['soil', 'treatment', 'seed_survival']]

# Convert categorical variables to dummy variables (one-hot encoding)
plant_growth_variables = pd.get_dummies(plant_growth_variables, columns=['soil', 'treatment'])

print(plant_growth_variables)

plant_growth_variables = plant_growth_variables.drop(['soil_Russell Ranch', 'treatment_Control'], axis=1)

plant_growth_variables = plant_growth_variables.astype('int32')

# Define independent variables (predictors)
independent_variables = plant_growth_variables.drop(columns=['seed_survival'])  # Drop the dependent variable
independent_variables = sm.add_constant(independent_variables)  # Add an intercept term


# Define dependent variable (response)
dependent_variable = plant_growth_variables['seed_survival']

# Fit logistic regression model
logistic_regression = sm.Logit(dependent_variable, independent_variables).fit()

# Print summary
print(logistic_regression.summary())

## Shoot length statistics

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd

plant_growth_variables = all_plant_growth_df[['soil', 'treatment', 'shoot_length_cm']]

# FIRST ATTEMPT: CONDUCTING SHOOT LENGTH TEST AS LINEAR REGRESSION WITH 1-HOT ENCODINGS

# # Convert categorical variables to dummy variables (one-hot encoding)
# plant_growth_variables = pd.get_dummies(plant_growth_variables, columns=['soil', 'treatment'])

# plant_growth_variables = plant_growth_variables.drop(['soil_Russell Ranch', 'treatment_Control'], axis=1)

# plant_growth_variables = plant_growth_variables.astype('int32')

# # Define independent variables (predictors)
# independent_variables = plant_growth_variables.drop(columns=['shoot_length_cm'])  # Drop the dependent variable
# independent_variables = sm.add_constant(independent_variables)  # Add an intercept term

# # Define dependent variable (response)
# dependent_variable = plant_growth_variables['shoot_length_cm']

# result = sm.OLS(dependent_variable, independent_variables).fit()
# print(result.summary())


# SECOND ATTEMPT: CONDUCTING SHOOT LENGTH TEST USING 2 WAY ANOVA

# Fit two-way ANOVA model
model = smf.ols('shoot_length_cm ~ C(soil) + C(treatment) + C(soil):C(treatment)', data=plant_growth_variables).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model, typ=2)
print(anova_results)


# Combine soil and treatment into a single factor for comparison
plant_growth_variables.loc[:, 'soil_treatment'] = plant_growth_variables['soil'].astype(str) + " - " + plant_growth_variables['treatment'].astype(str)

# Perform Tukey's HSD
tukey = pairwise_tukeyhsd(plant_growth_variables['shoot_length_cm'], plant_growth_variables['soil_treatment'])
print(tukey)


## Spikelet Count Statistics

In [ ]:
import statsmodels.formula.api as smf

plant_growth_variables = all_plant_growth_df[['soil', 'treatment', 'num_spikelets']]
print(plant_growth_variables)

plant_growth_variables['soil'] = pd.Categorical(plant_growth_variables['soil'], categories=['Russell Ranch', 'Manteca'], ordered=False)
plant_growth_variables['treatment'] = pd.Categorical(plant_growth_variables['treatment'], categories=['Control', 'Compost', 'Biochar-Compost'], ordered=False)


model = smf.glm(formula="num_spikelets ~ soil + treatment", 
                data=plant_growth_variables, 
                family=sm.families.Poisson()).fit()
print(model.summary())

pearson_chi2 = model.pearson_chi2
df_residual = model.df_resid
dispersion = pearson_chi2 / df_residual
print(f"Dispersion: {dispersion:.2f}")  # If >1, consider Negative Binomial

plant_growth_variables.loc[:, 'soil_treatment'] = plant_growth_variables['soil'].astype(str) + " - " + plant_growth_variables['treatment'].astype(str)

# Perform Tukey's HSD
tukey = pairwise_tukeyhsd(plant_growth_variables['num_spikelets'], plant_growth_variables['soil_treatment'])
print(tukey)

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.discrete.count_model import ZeroInflatedPoisson

plant_growth_variables = all_plant_growth_df[['soil', 'treatment', 'num_spikelets']]
print(plant_growth_variables)

plant_growth_variables['soil'] = pd.Categorical(plant_growth_variables['soil'], categories=['Russell Ranch', 'Manteca'], ordered=False)
plant_growth_variables['treatment'] = pd.Categorical(plant_growth_variables['treatment'], categories=['Control', 'Compost', 'Biochar-Compost'], ordered=False)


zip_model = ZeroInflatedPoisson.from_formula(
    "num_spikelets ~ soil + treatment",  # Main Poisson regression
    data=plant_growth_variables
).fit()

print(zip_model.summary())

mu = zip_model.predict()
pearson_chi2 = ((plant_growth_variables["num_spikelets"] - mu) ** 2 / mu).sum()
dispersion_ratio = pearson_chi2 / (len(plant_growth_variables) - zip_model.df_model)
print("Dispersion Ratio:", dispersion_ratio)

In [ ]:
sns.histplot(end_chem_properties_df['pH'], kde=True, bins=10, color='blue')

In [ ]:
import scipy.stats as stats

stats.probplot(end_chem_properties_df['pH'], dist="norm", plot=plt)
plt.show()

In [ ]:
import pandas as pd
import scipy.stats as stats

# Function to run Shapiro-Wilk test for each group
def check_normality(data):
    groups = data.groupby(['Soil', 'Treatment', 'Time'])
    results = {}
    for name, group in groups:
        stat, p = stats.shapiro(group['pH'])  # Run Shapiro-Wilk test
        results[name] = p
    return pd.DataFrame.from_dict(results, orient='index', columns=['p-value'])

# Apply function
normality_results = check_normality(chem_properties_df)
print(normality_results)

## This means that **normality is not violated**!!! (since p > 0.05)

In [ ]:
import pingouin as pg

sphericity = pg.sphericity(chem_properties_df, dv='pH', within='Time', subject='Sample')
print(sphericity)

# pH and EC measurements

In [ ]:
RR_chem_properties_df = Table().read_table('RR_Chemical_Properties.csv').to_df()

Man_chem_properties_df = Table().read_table('Man_Chemical_Properties.csv').to_df()

RR_chem_properties_df['category'] = RR_chem_properties_df['Soil'] + '_' + RR_chem_properties_df['Treatment']
Man_chem_properties_df['category'] = Man_chem_properties_df['Soil'] + '_' + Man_chem_properties_df['Treatment']

chem_properties_df = pd.concat([RR_chem_properties_df, Man_chem_properties_df])

t0_chem_properties_df = chem_properties_df.where(chem_properties_df['Time'] == "T0").dropna()
end_chem_properties_df = chem_properties_df.where(chem_properties_df['Time'] == "end").dropna()
# RR_ph_averages_df = RR_chem_properties_df[['Treatment', 'pH', 'Time']]
# RR_ph_averages_df.groupby(['Treatment', 'Time']).mean()
# RR_ph_diffs_df = RR_ph_averages_df['Treatment']

In [ ]:
t0_avg = t0_chem_properties_df.groupby('category')[['pH', 'EC (uS/cm)']].mean()
t0_avg = t0_avg.rename(columns={'pH': 'Avg_T0_pH', 'EC (uS/cm)': 'Avg_T0_EC'})

end_chem_properties_df = end_chem_properties_df.merge(t0_avg, on='category')
end_chem_properties_df["delta_pH"] = end_chem_properties_df["pH"] - end_chem_properties_df["Avg_T0_pH"]
end_chem_properties_df["delta_EC"] = end_chem_properties_df["EC (uS/cm)"] - end_chem_properties_df["Avg_T0_EC"]

## Check for normality

In [ ]:
# Check normality assumption with Shapiro-Wilk test for each Soil-Treatment combination
print("Normality Test (Shapiro-Wilk p-values)")
normality_data = {"Soil": [], "Treatment": [], "p-value": []}
for (soil, treatment), group in end_chem_properties_df.groupby(["Soil", "Treatment"]):
    stat, p_value = stats.shapiro(group["delta_pH"])
    normality_data['Soil'].append(soil)
    normality_data['Treatment'].append(treatment)
    normality_data['p-value'].append(p_value)

print(pd.DataFrame(normality_data))

The result above shows us that our data is normal. Across all of the p-values, we see that the data cannot reject the null hypothesis, and therefore shows normality.

## Check for Homogeneity of Variance using Bartlett's Test

In [ ]:
# Check homogeneity of variance (Bartlett's test)
bartlett_stat, bartlett_p = stats.bartlett(
    *[group["delta_pH"].values for _, group in end_chem_properties_df.groupby(["Soil", "Treatment"])]
)
print(f"Bartlett's Test results in a p-value of {bartlett_p}")

Since this value is less than 0.05, we reject the null hypothesis, that the variances are similar. This means that we cannot follow a simple ANOVA procedure to calculate significance of our data. Instead, we will have to use an alternative strategy, like Welch's ANOVA. 

## Conduct Welch's ANOVA

In [ ]:
import pingouin as pg
welch_anova_results = pg.welch_anova(dv="delta_pH", between=["Soil", "Treatment"], data=end_chem_properties_df)